In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
import os
import pandas as pd
pd.set_option('display.max_columns',40)

# Mount Google Drive
#drive.mount('/content/drive')

# Path to your data folder in Google Drive
data_folder = '/content/drive/MyDrive/Real_Trader_EDA_R1/240_Minute_Data/Calculate_240_Min_Data_folders/Equal_Distance_Based_ESDT_Data'

# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder)  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['ESDT_ZB 09-24_Minute_240.csv',
 'ESDT_NQ 09-24_Minute_240.csv',
 'ESDT_ES 09-24_Minute_240.csv']

In [3]:
import os
import pandas as pd

# Define the function to update the DataFrame
def update_new_target(df):
    # Convert 'Time Stamp' column to datetime
    if 'Time Stamp' in df.columns:
        df['Time Stamp'] = pd.to_datetime(df['Time Stamp'])

    # Ensure numeric columns are actually numeric
    columns_to_convert = ['Entry', 'Stop', 'BuyersInControlPrice', 'SellersInControlPrice',
                          'SellersLevel4', 'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
                          'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3', 'BuyersLevel4']

    df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

    # Create a new column for ED_Target
    df['ED_Target'] = 0  # Initialize with 0

    def update_entry(row):
        if pd.isna(row['Entry']) or row['Entry'] == 0:
            return 0  # Set ED_Target to 0 if Entry is 0

        # Check BuyersInControlPrice condition
        if row['BuyersInControlPrice'] != 0:
            updated_entry = row['Entry'] + (row['Entry'] - row['Stop'])
            # Find values greater than updated_entry and sort them
            valid_values = sorted([row[col] for col in ['SellersLevel4', 'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
                                                         'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3', 'BuyersLevel4']
                                   if pd.notna(row[col]) and row[col] > updated_entry])

            if valid_values:
                return valid_values[0]  # Return the smallest value that is greater than updated_entry
            return updated_entry  # Return updated_entry if no valid values are found

        # Check SellersInControlPrice condition
        elif row['SellersInControlPrice'] != 0:
            updated_entry = row['Entry'] - (row['Stop'] - row['Entry'])
            # Find values less than updated_entry and sort them
            valid_values = sorted([row[col] for col in ['SellersLevel4', 'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
                                                         'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3', 'BuyersLevel4']
                                   if pd.notna(row[col]) and row[col] < updated_entry], reverse=True)

            if valid_values:
                return valid_values[0]  # Return the largest value that is less than updated_entry
            return updated_entry  # Return updated_entry if no valid values are found

        return 0

    # Apply the update_entry function to each row
    df['ED_Target'] = df.apply(update_entry, axis=1)
    return df

def process_files_in_directory(input_directory, output_directory):
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Get all CSV files in the input directory
    files = [f for f in os.listdir(input_directory) if f.endswith('.csv')]

    for file in files:
        file_path = os.path.join(input_directory, file)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Drop the specified columns if they exist
        columns_to_drop = ['Unnamed: 0', 'Target']
        df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

        # Apply the trading feature generation function
        df = update_new_target(df)

        # Save the modified DataFrame to the output directory with prefix 'Updated_Target_'
        output_file_name = f"Updated_Target_{file}"
        output_file_path = os.path.join(output_directory, output_file_name)
        df.to_csv(output_file_path, index=False)

        print(f"Processed and saved {output_file_name} to {output_directory}")

In [4]:
output_directory = '/content/drive/MyDrive/Real_Trader_EDA_R1/240_Minute_Data/Calculate_240_Min_Data_folders/Updated_Target_Equasl_Distance_Based_Data'
process_files_in_directory(data_folder, output_directory)

Processed and saved Updated_Target_ESDT_ZB 09-24_Minute_240.csv to /content/drive/MyDrive/Real_Trader_EDA_R1/240_Minute_Data/Calculate_240_Min_Data_folders/Updated_Target_Equasl_Distance_Based_Data
Processed and saved Updated_Target_ESDT_NQ 09-24_Minute_240.csv to /content/drive/MyDrive/Real_Trader_EDA_R1/240_Minute_Data/Calculate_240_Min_Data_folders/Updated_Target_Equasl_Distance_Based_Data
Processed and saved Updated_Target_ESDT_ES 09-24_Minute_240.csv to /content/drive/MyDrive/Real_Trader_EDA_R1/240_Minute_Data/Calculate_240_Min_Data_folders/Updated_Target_Equasl_Distance_Based_Data
